In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from typing import *
import time
import math
import random
import wandb
wandb.login()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# define Lang
class Lang:
	def __init__(self, wordList):
		self.char2index = {'A': 0, 'Z': 1, '_': 2, '^':3}
		self.char2count = {}
		self.index2char = {0: 'A', 1: 'Z', 2: '_', 3: '^'}
		self.n_chars = 4

		for word in wordList:
			self.addWord(word)

	def addWord(self, word):
		for char in word:
			self.addChar(char)

	def addChar(self, char):
		if char not in self.char2index:
			self.char2index[char] = self.n_chars
			self.char2count[char] = 1
			self.index2char[self.n_chars] = char
			self.n_chars += 1
		elif char != '_' and char != '^' and char != 'A' and char != 'Z':
			self.char2count[char] += 1

	def encode(self, word):
		encoded = [0] * len(word)
		for i in range(len(word)):
			if word[i] in self.char2index:
				encoded[i] = self.char2index[word[i]]
			else:
				encoded[i] = self.char2index['^']
		return encoded
	
	def one_hot_encode(self, word):
		one_hot = torch.zeros(len(word), self.n_chars, device=device)
		for i in range(len(word)):
			if word[i] in self.char2index:
				one_hot[i][self.char2index[word[i]]] = 1
			else:
				one_hot[i][self.char2index['^']] = 1			
		return one_hot
		
	def decode(self, word):
		decoded = ''
		for i in range(len(word)):
			if word[i].argmax().item() in self.index2char:
				decoded += self.index2char[word[i].argmax().item()]
			else:
				decoded += '^'
		return decoded
	
	def decode_words(self, words):
		n_words = words.argmax(dim=2).T
		decoded = ['']*len(n_words)
		for i in range(len(n_words)):
			decoded[i] = ''.join([self.index2char[n_words[i][j].item()] for j in range(len(n_words[i]))])

		return decoded
	
	def decode_one_hot(self, word):
		decoded = ''
		for i in range(len(word)):
			if word[i].argmax().item() in self.index2char:
				decoded += self.index2char[word[i].argmax().item()]
			else:
				decoded += '^'
		return decoded

In [18]:
def tensorFromWord(lang : Lang, word : str):
	indexes = lang.encode(word)
	return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromWords(lang : Lang, words : List[str]):
	tensors = []
	for word in words:
		tensors.append(tensorFromWord(lang, word))
	return torch.cat(tensors, dim=1)

def tensorsFromPair(pair, inp_lang : Lang, out_lang : Lang):
	input_tensor = tensorFromWord(inp_lang, pair[0])
	target_tensor = tensorFromWord(out_lang, pair[1])
	return (input_tensor.unsqueeze(1), target_tensor)

def tensorsFromPairs(pairs, inp_lang : Lang, out_lang : Lang, batch_size):
	tensors_inp = []
	tensors_out = []
	for pair in pairs:
		tensors_inp.append(tensorFromWord(inp_lang, pair[0]))
		tensors_out.append(tensorFromWord(out_lang, pair[1]))
	return torch.cat(tensors_inp, dim=1).view(-1,1,batch_size), torch.cat(tensors_out, dim=1)

In [19]:
def DataLoader(lang : str, pad : bool = False, max_length : int = 40):
	train_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_train.csv')
	test_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_test.csv')
	valid_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_valid.csv')
	# train_data = pd.read_csv('/kaggle/input/gujarati/guj_train.csv')
	# test_data = pd.read_csv('/kaggle/input/gujarati/guj_test.csv')
	# valid_data = pd.read_csv('/kaggle/input/gujarati/guj_valid.csv')

	train_data.columns = ['input_seq', 'target_seq']
	test_data.columns = ['input_seq', 'target_seq']
	valid_data.columns = ['input_seq', 'target_seq']
        
	train_data['input_seq'] = train_data['input_seq'].apply(lambda x: x + 'Z')
	train_data['target_seq'] = train_data['target_seq'].apply(lambda x: x + 'Z')
	test_data['input_seq'] = test_data['input_seq'].apply(lambda x: x + 'Z')
	test_data['target_seq'] = test_data['target_seq'].apply(lambda x: x + 'Z')
	valid_data['input_seq'] = valid_data['input_seq'].apply(lambda x: x + 'Z')
	valid_data['target_seq'] = valid_data['target_seq'].apply(lambda x: x + 'Z')

	if pad:
		train_data['input_seq'] = train_data['input_seq'].apply(lambda x: x + '_'*(max_length - len(x)))
		train_data['target_seq'] = train_data['target_seq'].apply(lambda x: x + '_'*(max_length - len(x)))
		test_data['input_seq'] = test_data['input_seq'].apply(lambda x: x + '_'*(max_length - len(x)))
		test_data['target_seq'] = test_data['target_seq'].apply(lambda x: x + '_'*(max_length - len(x)))
		valid_data['input_seq'] = valid_data['input_seq'].apply(lambda x: x + '_'*(max_length - len(x)))
		valid_data['target_seq'] = valid_data['target_seq'].apply(lambda x: x + '_'*(max_length - len(x)))

	return train_data, test_data, valid_data

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [20]:
def get_cell(str):
	if str == 'lstm':
		return nn.LSTM
	elif str == 'gru':
		return nn.GRU
	elif str == 'rnn':
		return nn.RNN
	else:
		raise ValueError('Invalid cell type')

In [47]:
class EncoderRNN(nn.Module):
	def __init__(self, input_size, embed_size, hidden_size, n_layers=1, type='gru', dropout=0.2, batch_size=1):
		super(EncoderRNN, self).__init__()
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.type_t = type
		self.batch_size = batch_size

		self.embedding = nn.Embedding(input_size, embed_size)
		self.cell = get_cell(type)(embed_size, hidden_size, n_layers, dropout=dropout)

	def forward(self, input, hidden):
		embedded = self.embedding(input)
		output = embedded
		output, hidden = self.cell(output, hidden)
		return output, hidden

	def initHidden(self, bs = -1):
		if bs == -1:
			bs = self.batch_size
		if self.type_t == 'lstm':
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device), torch.zeros(self.n_layers, bs, self.hidden_size, device=device)
		else:
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device)
	
class DecoderRNN(nn.Module):
	def __init__(self, hidden_size, output_size, n_layers=1, type='gru', dropout=0.2, batch_size=1):
		super(DecoderRNN, self).__init__()
		self.hidden_size = hidden_size
		self.n_layers = n_layers
		self.type_t = type
		self.batch_size = batch_size

		self.embedding = nn.Embedding(output_size, hidden_size)
		self.cell = get_cell(type)(hidden_size, hidden_size, n_layers, dropout=dropout)
		self.out = nn.Linear(hidden_size, output_size)
		self.softmax = nn.LogSoftmax(dim=1)

	def forward(self, input, hidden, bs = -1):
		if bs == -1:
			bs = self.batch_size
		output = self.embedding(input).view(1, bs, -1)
		output = F.relu(output)
		output, hidden = self.cell(output, hidden)
		output = self.softmax(self.out(output[0]))
		return output, hidden

	def initHidden(self, bs = -1):
		if bs == -1:
			bs = self.batch_size
		if self.type_t == 'lstm':
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device), torch.zeros(self.n_layers, bs, self.hidden_size, device=device)
		else:
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device)
		

class AttnDecoderRNN(nn.Module):
	def __init__(self, hidden_size, output_size, n_layers=1, type='gru', dropout_p=0.1, batch_size=1, max_length=50):
		super(AttnDecoderRNN, self).__init__()
		self.hidden_size = hidden_size
		self.output_size = output_size
		self.dropout_p = dropout_p
		self.max_length = max_length
		self.n_layers = n_layers
		self.type_t = type
		self.batch_size = batch_size

		self.cell = get_cell(type)(hidden_size, hidden_size, n_layers, dropout=dropout_p)
		self.embedding = nn.Embedding(self.output_size, self.hidden_size)
		self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
		self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
		self.dropout = nn.Dropout(self.dropout_p)
		self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers, dropout=self.dropout_p)
		self.out = nn.Linear(self.hidden_size, self.output_size)

	def forward(self, input, hidden, encoder_outputs, bs=-1):
		if bs == -1:
			bs = self.batch_size
		embedded = self.embedding(input).view(1, bs, -1)
		embedded = self.dropout(embedded)

		if self.type_t != 'lstm':
			attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
		else:
			attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
		attn_applied = torch.bmm(attn_weights.unsqueeze(0),
								encoder_outputs.unsqueeze(0))

		output = torch.cat((embedded[0], attn_applied[0]), 1)
		output = self.attn_combine(output).unsqueeze(0)

		output = F.relu(output)
		output, hidden = self.cell(output, hidden)

		output = F.log_softmax(self.out(output[0]), dim=1)
		return output, hidden, attn_weights

	def initHidden(self, bs=-1):
		if bs == -1:
			bs = self.batch_size
		if self.type_t == 'lstm':
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device), torch.zeros(self.n_layers, bs, self.hidden_size, device=device)
		else:
			return torch.zeros(self.n_layers, bs, self.hidden_size, device=device)

In [48]:
class Seq2Seq(nn.Module):
	def __init__(self, input_size, hidden_size, embed_size, output_size, enc_layers=1, dec_layers=1, type='gru', dropout=0.2, batch_size=1, max_length=50,is_attn=False):
		super(Seq2Seq, self).__init__()
		self.input_size = input_size
		self.hidden_size = hidden_size
		self.output_size = output_size
		self.enc_layers = enc_layers
		self.dec_layers = dec_layers
		self.batch_size = batch_size
		self.dropout = dropout
		self.max_length = max_length
		self.is_attn = is_attn
		self.type_t = type

		self.encoder = EncoderRNN(input_size, embed_size, hidden_size, enc_layers, type, dropout, batch_size).to(device)
		if not is_attn:
			self.decoder = DecoderRNN(hidden_size, output_size, dec_layers, type, dropout, batch_size).to(device)
		else:
			self.decoder = AttnDecoderRNN(hidden_size, output_size, dec_layers, type, dropout, batch_size, max_length).to(device)

	def forward(self, input_tensor, target_tensor):
		bs = input_tensor.size(2)
		encoder_hidden = self.encoder.initHidden(bs)

		input_length = input_tensor.size(0)
		target_length = target_tensor.size(0)

		encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

		for ei in range(input_length):
			encoder_output, encoder_hidden = self.encoder(
				input_tensor[ei], encoder_hidden)
			encoder_outputs[ei] = encoder_output[0, 0]

		lst = []
		for i in range(bs):
			lst.append([0])
		decoder_input = torch.tensor(lst, device=device)  # SOS

		if self.encoder.n_layers == self.decoder.n_layers:
			decoder_hidden = encoder_hidden
		elif self.type_t != 'lstm':
			decoder_hidden = torch.zeros(self.decoder.n_layers, bs, self.decoder.hidden_size, device=device)
			av = encoder_hidden[0]
			for i in range(1,self.encoder.n_layers):
				av += encoder_hidden[i]
			av /= self.encoder.n_layers
			for i in range(self.decoder.n_layers):
				decoder_hidden[i] = av
		else:
			decoder_hidden = (torch.zeros(self.decoder.n_layers, bs, self.decoder.hidden_size, device=device), torch.zeros(self.decoder.n_layers, bs, self.decoder.hidden_size, device=device))
			av = encoder_hidden[0][0]
			for i in range(1,self.encoder.n_layers):
				av += encoder_hidden[0][i]
			av /= self.encoder.n_layers
			for i in range(self.decoder.n_layers):
				decoder_hidden[0][i] = av
			av = encoder_hidden[1][0]
			for i in range(1,self.encoder.n_layers):
				av += encoder_hidden[1][i]
			av /= self.encoder.n_layers
			for i in range(self.decoder.n_layers):
				decoder_hidden[1][i] = av

		use_teacher_forcing = True if random.random() < 0.5 else False

		decoder_outputs = torch.zeros(target_length, bs, self.output_size, device=device)
		if not self.is_attn:
			if use_teacher_forcing:
				# Teacher forcing: Feed the target as the next input
				for di in range(target_length):
					decoder_output, decoder_hidden = self.decoder(
						decoder_input, decoder_hidden, bs)
					decoder_outputs[di] = decoder_output
					decoder_input = target_tensor[di]  # Teacher forcing
			else:
				# Without teacher forcing: use its own predictions as the next input
				for di in range(target_length):
					decoder_output, decoder_hidden = self.decoder(
						decoder_input, decoder_hidden, bs)
					decoder_outputs[di] = decoder_output
					topv, topi = decoder_output.topk(1)
					decoder_input = topi.squeeze().detach()  # detach from history as input
					if self.batch_size == 1:
						if topi == 1:
							break
		else:
			if use_teacher_forcing:
				# Teacher forcing: Feed the target as the next input
				for di in range(target_length):
					decoder_output, decoder_hidden, decoder_attention = self.decoder(
						decoder_input, decoder_hidden, encoder_outputs, bs)
					decoder_outputs[di] = decoder_output
					decoder_input = target_tensor[di]
			else:
				# Without teacher forcing: use its own predictions as the next input
				for di in range(target_length):
					decoder_output, decoder_hidden, decoder_attention = self.decoder(
						decoder_input, decoder_hidden, encoder_outputs, bs)
					decoder_outputs[di] = decoder_output
					topv, topi = decoder_output.topk(1)
					decoder_input = topi.squeeze().detach()
					if self.batch_size == 1:
						if topi == 1:
							break

		return decoder_outputs

	def predict(self, input_tensor):
		encoder_hidden = self.encoder.initHidden(1)

		input_length = input_tensor.size(0)

		encoder_outputs = torch.zeros(self.max_length, self.encoder.hidden_size, device=device)

		for ei in range(input_length):
			encoder_output, encoder_hidden = self.encoder(
				input_tensor[ei], encoder_hidden)
			encoder_outputs[ei] = encoder_output[0, 0]

		decoder_input = torch.tensor([[0]], device=device)  # SOS

		decoder_outputs = []

		decoder_hidden = encoder_hidden
		if not self.is_attn:
			for di in range(self.max_length):
				decoder_output, decoder_hidden = self.decoder(
					decoder_input, decoder_hidden, 1)
				topv, topi = decoder_output.data.topk(1)
				if topi == 1:
					break
				decoder_outputs.append(decoder_output)
				decoder_input = topi.squeeze().detach()
		else:
			for di in range(self.max_length):
				decoder_output, decoder_hidden, decoder_attention = self.decoder(
					decoder_input, decoder_hidden, encoder_outputs, 1)
				topv, topi = decoder_output.data.topk(1)
				if topi == 1:
					break
				decoder_outputs.append(decoder_output)
				decoder_input = topi.squeeze().detach()

		return decoder_outputs

In [49]:
class Translator:
	def __init__(self, lang, embed_size=10, hidden_size=10, enc_layers=1, dec_layers=1, max_length=50, type='gru', dropout=0.2, batch_size=1, is_attn=False):
		if batch_size != 1:
			self.train_data, self.test_data, self.valid_data = DataLoader(lang, pad = True, max_length = max_length)
		else:
			self.train_data, self.test_data, self.valid_data = DataLoader(lang, pad = False, max_length = max_length)

		self.inp_lang = Lang(self.train_data['input_seq'])
		self.out_lang = Lang(self.train_data['target_seq'])
		self.batch_size = batch_size

		self.model = Seq2Seq(self.inp_lang.n_chars, hidden_size, embed_size, self.out_lang.n_chars, enc_layers, dec_layers, type, dropout, batch_size, max_length, is_attn)
		self.criterion = nn.NLLLoss()
		self.max_length = max_length
		self.encoder_optim = optim.SGD(self.model.encoder.parameters(), lr=0.001)
		self.decoder_optim = optim.SGD(self.model.decoder.parameters(), lr=0.001)

		ps = [(self.train_data['input_seq'][i], self.train_data['target_seq'][i]) for i in range(len(self.train_data))]
		left = len(ps) % batch_size
		if left != 0:
			ps = ps[:-left]
		self.pairs = [tensorsFromPairs(ps[x:x+batch_size], self.inp_lang, self.out_lang, self.batch_size) for x in range(0, len(ps), batch_size)]

	def trainOne(self, input_tensor, target_tensor):
		self.encoder_optim.zero_grad()
		self.decoder_optim.zero_grad()

		bs = input_tensor.size(2)

		decoder_outputs = self.model.forward(input_tensor, target_tensor)

		loss = 0
		if self.batch_size != 1:
			for i in range(bs):
				loss1 = 0
				for j in range(len(decoder_outputs)):
					loss1 += self.criterion(decoder_outputs[j][i], target_tensor[j][i]) / bs
					if target_tensor[j][i].item() == 1:
						loss += loss1 * self.max_length / (j+1)
						loss1 = 0
						break
				loss += loss1
		else:
			for di in range(len(decoder_outputs)):
				loss += self.criterion(decoder_outputs[di], target_tensor[di]) 
		loss.backward()

		self.encoder_optim.step()
		self.decoder_optim.step()

		return loss.item() / target_tensor.size(0)

	def train(self,epoch=1, n_iters=10000, print_every=1000, plot_every=100, learning_rate=0.01, rand=False, dumpName='model', log=False, wandb = None):
		self.encoder_optim = optim.SGD(self.model.encoder.parameters(), lr=learning_rate)
		self.decoder_optim = optim.SGD(self.model.decoder.parameters(), lr=learning_rate)

		start = time.time()
		train_loss = []
		train_acc = []
		valid_loss = []
		valid_acc = []

		for i in range(epoch):
			print_loss_total = 0
			tot_loss = 0
			print("Epoch: ", i)
			if rand:
				training_pairs = [random.choice(self.pairs) for i in range(n_iters)]
			else:
				training_pairs = self.pairs

			for iter in tqdm(range(1, len(training_pairs) + 1)):
				training_pair = training_pairs[iter - 1]
				input_tensor = training_pair[0]
				target_tensor = training_pair[1]

				loss = self.trainOne(input_tensor, target_tensor)
				print_loss_total += loss 
				tot_loss += loss

				if iter % print_every == 0:
					print_loss_avg = print_loss_total / print_every
					print_loss_total = 0
					print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
												iter, iter / n_iters * 100, print_loss_avg))
			train_loss.append(tot_loss / len(training_pairs))
			train_acc.append(self.accuracy(self.train_data))
			valid_stats = self.calculate_stats(self.valid_data)
			valid_loss.append(valid_stats[0])
			valid_acc.append(valid_stats[1])
			print("Train Loss: ", train_loss[-1], "Valid Loss: ", valid_loss[-1], "Train Acc: ", train_acc[-1], "Valid Acc: ", valid_acc[-1])
			pickle.dump(self, open(dumpName + '_'+str(i) + '.pkl', 'wb'))
			if log:
				wandb.log({'train_loss': train_loss[i], 'train_accuracy': train_acc[i], 'val_loss': valid_loss[i], 'val_accuracy': valid_acc[i]})
		return train_loss, train_acc, valid_loss, valid_acc

	def accuracy(self, data):
		with torch.no_grad():
			acc = 0
			if self.batch_size == 1:
				for i in range(0, len(data), self.batch_size):
					tensors = tensorsFromPairs([(data['input_seq'][j], data['target_seq'][j]) for j in range(i, i + self.batch_size)], self.inp_lang, self.out_lang, self.batch_size)
					outputs = self.model.forward(tensors[0], tensors[1])
					words = self.out_lang.decode_words(outputs)
					acc += np.sum([words[j].strip('Z') == data['target_seq'][i + j].strip('Z_') for j in range(self.batch_size)])
			else:
				tensors = tensorsFromPairs([(data['input_seq'][j], data['target_seq'][j]) for j in range(len(data))], self.inp_lang, self.out_lang, len(data))
				outputs = self.model.forward(tensors[0], tensors[1])
				words = self.out_lang.decode_words(outputs)
				acc += np.sum([words[j].strip('Z') == data['target_seq'][j].strip('Z_') for j in range(len(data))])
			return acc / len(data)

	def calculate_stats(self, data):
		with torch.no_grad():
			loss = 0
			acc = 0
			if self.batch_size == 1:
				for i in range(0, len(data), self.batch_size):
					tensors = tensorsFromPairs([(data['input_seq'][j], data['target_seq'][j]) for j in range(i, i + self.batch_size)], self.inp_lang, self.out_lang, self.batch_size)
					outputs = self.model.forward(tensors[0], tensors[1])
					for di in range(len(outputs)):
						loss += self.criterion(outputs[di], tensors[1][di]) / tensors[1].size(0)
					words = self.out_lang.decode_words(outputs)
					acc += np.sum([words[j].strip('Z') == data['target_seq'][i + j].strip('Z_') for j in range(self.batch_size)])
			else:
				tensors = tensorsFromPairs([(data['input_seq'][j], data['target_seq'][j]) for j in range(len(data))], self.inp_lang, self.out_lang, len(data))
				outputs = self.model.forward(tensors[0], tensors[1])
				for i in range(len(data)):
					loss1 = 0
					for j in range(len(outputs)):
						loss1 += self.criterion(outputs[j][i], tensors[1][j][i])
						if tensors[1][j][i].item() == 1:
							loss += loss1 / (j+1)
							loss1 = 0
							break
					loss += loss1 / len(outputs)
				words = self.out_lang.decode_words(outputs)
				acc += np.sum([words[j].strip('Z') == data['target_seq'][j].strip('Z_') for j in range(len(data))])
			return loss.item() / len(data), acc / len(data)

	def translate(self, word):
		with torch.no_grad():
			if self.batch_size != 1:
				word = word + 'Z' + '_'*(self.max_length - len(word) -1)
			tensor = tensorFromWord(self.inp_lang, word).unsqueeze(1)
			outs = self.model.predict(tensor)
			return self.out_lang.decode(outs)

In [50]:
sweep_configuration_new = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
    'parameters': 
    {
        'epochs' : {'values': [10]},
        'optim' : {'values' : ['SGD']},
        'lr' : {'values': [0.005]},
        'bs' : {'values': [1]},
        'hidden_size' : {'values': [256]},
        'embed_size' : {'values': [16]},
        'enc_layers' : {'values': [3]},
        'dec_layers' : {'values': [3]},
        'type' : {'values': ['lstm']},
        'dropout' : {'values': [0]},
        'is_attn' : {'values': [False]},
    }
}

In [51]:
# f = open('configs', 'w')
# count = 0

# def get_name(config):
# 	global count
# 	count += 1
# 	f.write(str(count) + '_' + str(config) + '\n')
# 	f.flush()
# 	return 'optim_SGD'+'_lr'+str(config['lr'])+'_hidden_size'+str(config['hidden_size'])+'_embed_size'+str(config['embed_size'])+'_enc_layers'+str(config['enc_layers'])+'_dec_layers'+str(config['dec_layers'])+'_type'+str(config['type'])+'_dropout'+str(config['dropout'])+'_bs'+str(config['bs'])+'_is_attn'+str(config['is_attn'])

# # 1: Define objective/training function
# def objective(config):
# 	translator = Translator('guj', hidden_size=config.hidden_size, embed_size=config.embed_size, enc_layers=config.enc_layers, dec_layers=config.dec_layers, type=config.type, dropout=config.dropout, batch_size=config.bs, is_attn=config.is_attn)
# 	dump = 'trans'+'_'+str(config.hidden_size)+'_'+str(config.embed_size)+'_'+str(config.enc_layers)+'_'+str(config.dec_layers)+'_'+str(config.type)+'_'+str(config.dropout)
# 	train_loss, train_acc, valid_loss, valid_acc = translator.train(epoch=config.epochs, print_every=(10000//config.bs), learning_rate=config.lr, dumpName=dump, log=True, wandb=wandb)

# 	# for i in range(config.epochs):
# 	# 	wandb.log({'train_loss': train_loss[i], 'train_accuracy': train_acc[i], 'val_loss': valid_loss[i], 'val_accuracy': valid_acc[i]})

# def main():
# 	run = wandb.init(project='rnn-first-proj')
# 	run.name = get_name(wandb.config)
# 	objective(wandb.config)
# 	run.finish()

# # 2: Define the search space
# sweep_configuration = sweep_configuration_new

# # 3: Start the sweep
# sweep_id = wandb.sweep(sweep=sweep_configuration, project='rnn-first-proj')
# wandb.agent(sweep_id, function=main, count=1)

# f.close()

In [53]:
trans = Translator('guj', hidden_size=512, embed_size=32, enc_layers=2, dec_layers=3, type='gru', dropout=0, batch_size=1, max_length=40, is_attn=True)
trans.train(epoch=10, print_every=1000, learning_rate=0.001)

Epoch:  0


  0%|          | 9/51199 [00:02<3:32:35,  4.01it/s]


KeyboardInterrupt: 

In [ ]:
# trans.accuracy(trans.valid_data)

In [ ]:
# trans.translate('a')

In [ ]:
# trans = pickle.load(open('trans_128_64_1_gru_0.3_9.pkl', 'rb'))

In [ ]:
# trans.translate('tahir')

In [ ]:
# # create a seq2seq model using 2 RNNs
# class Seq2Seq(nn.Module):
# 	def __init__(self, input_size, hidden_size, output_size, n_layers=1):
# 		super(Seq2Seq, self).__init__()
# 		self.input_size = input_size
# 		self.hidden_size = hidden_size
# 		self.output_size = output_size
# 		self.n_layers = n_layers

# 		# encoder and decoder
# 		self.encoder = nn.RNN(input_size, hidden_size, n_layers)
# 		self.decoder = nn.RNN(hidden_size, hidden_size, n_layers)

# 		# linear layer to get output
# 		self.linear = nn.Linear(hidden_size, output_size)

# 	def forward(self, input, hidden):
# 		# encoder
# 		output, hidden = self.encoder(input, hidden)
		
# 		# decoder
# 		output, hidden = self.decoder(output, hidden)
		
# 		# get output
# 		output = self.linear(output)
# 		return output, hidden
	
# 	def predict(self, input, inp_lang, out_lang):
# 		out, hidden = self.forward(inp_lang.one_hot_encode(input).unsqueeze(1), self.init_hidden(1))
# 		return out_lang.decode_one_hot(out)
	
# 	def init_hidden(self, batch_size):
# 		return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
# class Translator:
# 	def __init__(self, lang):
# 		train_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_train.csv')
# 		test_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_test.csv')
# 		valid_data = pd.read_csv(f'aksharantar_sampled/{lang}/{lang}_valid.csv')

# 		train_data.columns = ['input_seq', 'target_seq']
# 		test_data.columns = ['input_seq', 'target_seq']
# 		valid_data.columns = ['input_seq', 'target_seq']

# 		self.inp_lang = Lang(train_data['input_seq'])
# 		self.out_lang = Lang(train_data['target_seq'])

# 		self.model = Seq2Seq(self.inp_lang.n_chars, 10, self.out_lang.n_chars, 1)
# 		self.criterion = nn.CrossEntropyLoss()
# 		self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

# 		train_dataset = AksharantarDataset(train_data, self.inp_lang, self.out_lang)
# 		test_dataset = AksharantarDataset(test_data, self.inp_lang, self.out_lang)
# 		valid_dataset = AksharantarDataset(valid_data, self.inp_lang, self.out_lang)

# 		self.train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# 		self.test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
# 		self.valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=True)

# 	def translate(self, word):
# 		return self.model.predict(word, self.inp_lang, self.out_lang)
	
# 	def train_one(self, inp, target):
# 		# zero gradients
# 		self.optimizer.zero_grad()
		
# 		# initialize hidden layer
# 		hidden = self.model.init_hidden(1)
		
# 		# get output
# 		output, hidden = self.model.forward(inp, hidden)
		
# 		mx_len = min(len(output), len(target))

# 		if(len(output) != len(inp)):
# 			print('lol')

# 		# append output and target with 'Z' to make them of mx_len
# 		while(len(output) < mx_len):
# 			output = torch.cat((output, self.out_lang.one_hot_encode_char('Z')), 0)

# 		while(len(target) < mx_len):
# 			target = torch.cat((target, self.out_lang.one_hot_encode_char('Z')), 0)
		
# 		# calculate loss 
# 		loss = self.criterion(torch.flatten(output[:mx_len], 0, 1), torch.flatten(target[:mx_len], 0, 1).max(1)[1])
			
# 		# backpropagate
# 		loss.backward()
		
# 		# update weights
# 		self.optimizer.step()

# 		return loss.data.item() / len(output)
	
# 	def train_epoch(self, data_loader, print_every=100):
# 		loss = 0
# 		for i_batch, sample_batched in tqdm(enumerate(data_loader)):
# 			loss += self.train_one(sample_batched['input_seq'][0], sample_batched['target_seq'][0])
# 			if i_batch % print_every == print_every-1:
# 				print(' Train Loss: ', loss / (i_batch+1))
# 				# print(' Valid Loss: ', self.calc_loss_full(self.valid_dataloader))
# 				# print(' Train Accuracy: ', self.calc_accuracy(self.train_dataloader))
# 				# print(' Valid Accuracy: ', self.calc_accuracy(self.valid_dataloader))
# 		return self.calc_loss_full(data_loader)
	
# 	def train(self, epochs, print_every=100):
# 		losses = []
# 		for epoch in range(epochs):
# 			print('Epoch ', epoch + 1)
# 			loss = self.train_epoch(self.train_dataloader, print_every)
# 			losses.append(loss)
# 		return losses
	
# 	def calc_loss_full(self, data_loader):
# 		loss = 0
# 		for i_batch, sample_batched in tqdm(enumerate(data_loader)):
# 			inp = sample_batched['input_seq'][0]
# 			target = sample_batched['target_seq'][0]
# 			hidden = self.model.init_hidden(1)
# 			output, hidden = self.model.forward(inp, hidden)
# 			mx_len = min(len(output), len(target))
# 			while(len(output) < mx_len):
# 				output = torch.cat((output, self.out_lang.one_hot_encode_char('Z')), 0)
# 			while(len(target) < mx_len):
# 				target = torch.cat((target, self.out_lang.one_hot_encode_char('Z')), 0)
# 			loss += self.criterion(torch.flatten(output[:mx_len], 0, 1), torch.flatten(target[:mx_len], 0, 1).max(1)[1])
# 		return loss / len(data_loader)
	
# 	def calc_accuracy(self, dataset):
# 		return np.sum([(self.translate(sample_batched['input_seq']) ==  sample_batched['target_seq']) for sample_batched in dataset]) / len(dataset)

In [ ]:
# guj_trans = Translator('guj')
# guj_trans.train(10, 1000)